In [1]:
simbolo = "BTG"
epochs = 50
batch_size = 32
learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo
seed = 10 

def crea_modello(giorni_previsione, n_targets, n_features):
    model = Sequential([
        LSTM(50, input_shape=(n_timesteps, n_features)),
        RepeatVector(giorni_previsione),
        LSTM(50, return_sequences=True, kernel_regularizer=l2(0)),
        #LSTM(50, return_sequences=True, kernel_regularizer=l2(0)),
        #Dropout(0.2),
        #LSTM(100, return_sequences=True, recurrent_initializer="glorot_uniform", kernel_initializer="uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #LSTM(50, recurrent_initializer="glorot_uniform", kernel_initializer="uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #Dense(64, activation='relu', kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        TimeDistributed(Dense(n_targets, activation='linear'))
    ])
    return model

def to_XY(dati_ticker, n_timesteps, giorni_previsione):
    features = dati_ticker[[
        "Close",
        "EMA_5", 
        "EMA_20", 
        "EMA_50",
        "Open",  
        "High",
        "Low",
        "Volume",
        "MACDh",
        "PSAR",
        "PSARaf",
        "SUPERT", 
        "TRIX"
    ]]
    targets = dati_ticker[[
        "EMA_5",
        "EMA_20", 
        "EMA_50"
    ]]
    col_features = {col: idx for idx, col in enumerate(features.columns)}
    col_targets = {col: idx for idx, col in enumerate(targets.columns)}

    i_tot = len(features) - n_timesteps - giorni_previsione + 1
    idx, X, Y = [], [], []  
    for i in range(i_tot):
        idx.append(features.index[i + n_timesteps - 1])
        X.append(features.iloc[i:i + n_timesteps])
        Y.append(targets.iloc[i + n_timesteps:i + n_timesteps + giorni_previsione])
        
    return idx, np.array(X), np.array(Y), col_features, col_targets

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()

In [2]:
print("Inizio")
import pandas as pd
import yfinance as yf
import numpy as np
import tensorflow as tf
import fx_com
import funzioni as fx
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from keras.regularizers import l1, l2

np.random.seed(seed)
tf.random.set_seed(seed)

pd.set_option("display.max_columns", None)
print("Download dati ticker")
ticker = yf.download(simbolo, start='2010-01-01', end='2019-12-31', progress=False)
ticker.index = ticker.index.date
ticker = fx.crea_indicatori(ticker)
ticker.dropna(axis=0, inplace=True)

print("Definizione features e target")
idx, X, Y, col_features, col_targets = to_XY(ticker, n_timesteps, giorni_previsione)
n_features = X.shape[2]
n_targets = Y.shape[2]

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization

print("Creazione modello")
model = crea_modello(giorni_previsione, n_targets, n_features)

print("Compilazione modello")
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])

print("Scaler")
n_samples = X.shape[0]
X = X.reshape((-1, n_timesteps * n_features))
Y = Y.reshape((-1, giorni_previsione * n_targets))

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)

X_scaler = clone(scaler)
Y_scaler = clone(scaler)
X_scaler.fit(X_train)
Y_scaler.fit(Y_train)
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.fit_transform(X_test)
Y_train = Y_scaler.fit_transform(Y_train)
Y_test = Y_scaler.fit_transform(Y_test)

X_train = X_train.reshape((-1, n_timesteps, n_features))
X_test = X_test.reshape((-1, n_timesteps, n_features))
Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))
Y_test = Y_test.reshape((-1, giorni_previsione, n_targets))

print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test))

print("Scrittura risultati su file parquet")
import os
test_loss, test_mae = model.evaluate(X_test, Y_test)
if os.path.exists('metriche.parquet'):
    metriche = pd.read_parquet("metriche.parquet")
else:
    metriche = pd.DataFrame()
nuova_riga_metriche = pd.DataFrame([{
    "epochs": epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "n_timesteps": n_timesteps,
    "giorni_previsione": giorni_previsione,
    "test_loss": test_loss,
    "test_mae": test_mae
}])
for i, layer in enumerate(model.layers):
    config = layer.get_config()
    kernel_regularizer_config = config.get('kernel_regularizer', None)
    str_temp = f"{type(layer).__name__}"    
    if 'units' in config:
        str_temp += f", units({config['units']})"
    if 'rate' in config:
        str_temp += f", rate({config['rate']})" 
    if kernel_regularizer_config:
        class_name = kernel_regularizer_config.get('class_name', None)
        if class_name:
            str_temp += f", reg({class_name})" 
    nuova_riga_metriche[f"Layer_{i+1}"] = str_temp
metriche = pd.concat([metriche, nuova_riga_metriche], ignore_index=True)
metriche.to_parquet("metriche.parquet")

Inizio
Download dati ticker
Definizione features e target
Creazione modello
Compilazione modello
Scaler
Addestramento modello epochs=50, batch_size=32
Epoch 1/50
57/57 [==============================] - 13s 75ms/step - loss: 0.2781 - mae: 0.3458 - val_loss: 0.0664 - val_mae: 0.1834
Epoch 2/50
57/57 [==============================] - 2s 32ms/step - loss: 0.0533 - mae: 0.1564 - val_loss: 0.0367 - val_mae: 0.1395
Epoch 3/50
57/57 [==============================] - 2s 32ms/step - loss: 0.0310 - mae: 0.1242 - val_loss: 0.0293 - val_mae: 0.1282
Epoch 4/50
57/57 [==============================] - 2s 32ms/step - loss: 0.0231 - mae: 0.1077 - val_loss: 0.0222 - val_mae: 0.1097
Epoch 5/50
57/57 [==============================] - 2s 32ms/step - loss: 0.0181 - mae: 0.0956 - val_loss: 0.0204 - val_mae: 0.1052
Epoch 6/50
57/57 [==============================] - 2s 33ms/step - loss: 0.0159 - mae: 0.0895 - val_loss: 0.0205 - val_mae: 0.1061
Epoch 7/50
57/57 [==============================] - 2s 32ms/st

In [3]:
metriche

,epochs,batch_size,learning_rate,n_timesteps,giorni_previsione,test_loss,test_mae,Layer_1,Layer_2,Layer_3,Layer_4,Layer_5
0,50,32,0.001,10,5,0.012836,0.065889,"LSTM, units(150)",RepeatVector,"LSTM, units(100), reg(L2)","LSTM, units(50), reg(L2)",TimeDistributed
1,50,32,0.001,10,5,0.017022,0.061662,"LSTM, units(150)",RepeatVector,"LSTM, units(100), reg(L2)",TimeDistributed,None
2,50,32,0.001,10,5,0.017216,0.059458,"LSTM, units(100)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
3,50,32,0.001,10,5,0.011389,0.057679,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
4,50,32,0.001,10,5,0.008494,0.053422,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
5,50,32,0.001,10,10,0.015444,0.070930,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
6,50,32,0.001,10,10,0.054612,0.117111,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
7,50,32,0.001,60,10,0.058806,0.121026,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
8,50,32,0.001,60,10,0.002626,0.035899,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None
9,50,32,0.001,60,10,0.004300,0.046639,"LSTM, units(50)",RepeatVector,"LSTM, units(50), reg(L2)",TimeDistributed,None


In [4]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo, start='2020-01-01', end='2023-12-31', progress=False)
#dati_previsione = pd.read_csv("BTG.csv", index_col=0)
#dati_previsione = dati_previsione.loc['2020-01-01':'2023-09-10']
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

idx_prev, X_prev, Y_prev, col_features, col_targets = to_XY(dati_previsione, n_timesteps, giorni_previsione)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
X_scaler_prev = clone(scaler)
Y_scaler_prev = clone(scaler)
X_prev = X_scaler_prev.fit_transform(X_prev)
Y_prev = Y_scaler_prev.fit_transform(Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

pred = model.predict(X_prev)
pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = X_scaler_prev.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)

Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
pred = pred.reshape(-1, giorni_previsione * n_targets)
Y_prev = Y_scaler_prev.inverse_transform(Y_prev)
pred = Y_scaler_prev.inverse_transform(pred)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)
pred = pred.reshape(-1, giorni_previsione, n_targets)



Download dati ticker per previsione
27/27 [==============================] - 0s 13ms/step - loss: 0.0546 - mae: 0.1644


In [5]:
col_analisi = "EMA_5"
risultato = pd.DataFrame({"Feature": X_prev[:, n_timesteps-1, col_features[col_analisi]], "Target": Y_prev[:, giorni_previsione-1, col_targets[col_analisi]].round(2), "Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=idx_prev)

fx_com.grafico(risultato, pred_mae)
#risultato.to_excel("ris.xlsx")